In [1]:
kernel.silent(true)

In [2]:
import coursierapi.MavenRepository
interp.repositories() ++= Seq(MavenRepository.of("https://jitpack.io"))

In [3]:
import $ivy.`com.github.propi:rdfrules:1.5.0`
import collection._
import org.apache.jena.riot.Lang

import com.github.propi.rdfrules.data._
import com.github.propi.rdfrules.algorithm.amie._
import com.github.propi.rdfrules.algorithm.dbscan._
import com.github.propi.rdfrules.utils._
import com.github.propi.rdfrules.index._
import com.github.propi.rdfrules.rule._
import com.github.propi.rdfrules.ruleset._

In [4]:
val facilitiesDataset = Dataset.fromCache("../cache/facilitiesCube.cache")
val wikidataDataset = Dataset.fromCache("../cache/wikidata.cache")

In [5]:
val refPeriodLinking = Dataset("../data/linking/refPeriodLinking.ttl")

In [13]:
val czsoWikidataLinking = Dataset("../data/linking/czsoWikidataLinking.ttl")

In [14]:
val dataset = facilitiesDataset + wikidataDataset + refPeriodLinking + czsoWikidataLinking

In [15]:
val index = dataset.index()

In [16]:
val uri = (value: String) => TripleItem.Uri(value)
val qbdPredicate = uri("http://purl.org/linked-data/cube#dataSet")
val czsoRefPeriod = uri("http://data.czso.cz/ontology/refPeriod")
val czsoRefArea = uri("http://data.czso.cz/ontology/refArea")
val oneOfMeasures = OneOf(
uri("http://data.czso.cz/ontology/pocetSocialnichZarizeni"),
uri("http://data.czso.cz/ontology/pocetMistVSocialnichZarizenich"),
uri("http://data.czso.cz/ontology/pocetDomovuProDuchodce"),
uri("http://data.czso.cz/ontology/pocetMistVDomovechProDuchodce"),
uri("http://data.czso.cz/ontology/pocetDomovuProOsobySeZdravotnimPostizenim"),
uri("http://data.czso.cz/ontology/pocetMistVDomovechProOsobySeZdravotnimPostizenim"),
uri("http://data.czso.cz/ontology/pocetAzylovychDomu"),
uri("http://data.czso.cz/ontology/pocetMistVAzylovychDomech"),
uri("http://data.czso.cz/ontology/pocetDomuSPecovatelskouSluzbou"),
uri("http://data.czso.cz/ontology/pocetBytovychJednotekVDomechSPecovatelskouSluzbou")
)

In [57]:
val facilitiesPattern: RulePattern = (
    AtomPattern(subject = 'c', `object` = 'b', graph = uri("wikidata")) &:
    //AtomPattern(subject = 'a', predicate = czsoRefPeriod, `object` = 'c') &:
    AtomPattern(subject = 'a', predicate = czsoRefArea, `object` = 'b') &:
    AtomPattern(subject = 'a', predicate = qbdPredicate, `object` = AnyConstant)
    =>:
    AtomPattern(subject = 'a', predicate = oneOfMeasures)
)

In [58]:
val facilitiesTask = Amie()
    .addThreshold(Threshold.MinSupport(1))
    .addThreshold(Threshold.MaxRuleLength(9))
    .addThreshold(Threshold.MinHeadSize(0))
    .addThreshold(Threshold.Timeout(1))
    .addPattern(facilitiesPattern)

In [59]:
val facilitiesTaskRuleset = index.mine(facilitiesTask)
println(facilitiesTaskRuleset.size)

2021-03-30 18:20:48:043 +0200 [scala-interpreter-1] INFO com.github.propi.rdfrules.utils.Debugger - Amie task settings:
MinHeadSize=1,
MinHeadCoverage=0.0,
MinSupport=1,
MaxThreads=4,
MinAtomSize=0,
MaxRuleLength=9,
WithConstants=true,
ConstantsPosition=All,
Timeout=60000,
WithDuplicitPredicates=true,
Patterns=List(Mapped(Vector(Mapped(Variable(?c),Any,Variable(?b),Constant(Constant(1350603828))), Mapped(Variable(?a),Constant(Constant(1481837794)),Variable(?b),Any), Mapped(Variable(?a),Constant(Constant(624690160)),AnyConstant,Any)),Some(Mapped(Variable(?a),OneOf(ArrayBuffer(Constant(Constant(-1020668445)), Constant(Constant(-394053961)), Constant(Constant(-970864269)), Constant(Constant(643010916)), Constant(Constant(-1550362221)), Constant(Constant(-1004845722)), Constant(Constant(-104573618)), Constant(Constant(1935867686)), Constant(Constant(420112347)), Constant(Constant(1862618459)))),Any,Any)),false,false)),
OnlyPredicates=None,
WithoutPredicates=None
2021-03-30 18:21:52:943 +02

45289


In [60]:
val ruleset = facilitiesTaskRuleset
ruleset
//.filter(rule => rule.measures(Measure.Support).value > 2)
//.computePcaConfidence(0.5)
.export("../rulesets/facilitiesTaskRuleset.txt")